DQN SERIA

In [ ]:
import time
from typing import Dict, Any
import math

In [ ]:
balas = []
def distancia_trayectoria(x0, y0, vx0, vy0, x_target, y_target):
    """Calcula la distancia mínima entre un punto (x_target, y_target) y una trayectoria lineal definida por posición (x0, y0) y velocidad (vx0, vy0)."""

    """
    """
    if vx0 == 0 and vy0 == 0:
        # Si no hay movimiento, calcula distancia euclidiana
        return math.sqrt((x0 - x_target)**2 + (y0 - y_target)**2)
    
    if vx0 != 0:
        # Caso no vertical: y = mx + b
        # 0 = mx + b - y
        m = vy0 / vx0
        b = y0 - m * x0
        dist_lagrima_enemigo = abs(m * x_target - y_target + b) / math.sqrt(m**2 + 1)
        pos_x_mas_cercana_al_enemigo, pos_y_mas_cercana_al_enemigo = posicion_de_la_recta_mas_cercana_al_enemigo(m, -1, b, x_target, y_target)
    else:
        # Caso vertical: x = constante
        dist_lagrima_enemigo = abs(x_target - x0)
        pos_x_mas_cercana_al_enemigo = x0
        if (vy0 < 0 and y_target <= y0) or (vy0 > 0 and y_target >= y0):
            pos_y_mas_cercana_al_enemigo = y_target
        else:
            pos_y_mas_cercana_al_enemigo = y0

    return dist_lagrima_enemigo, pos_x_mas_cercana_al_enemigo, pos_y_mas_cercana_al_enemigo

def rango_predicho(vx, vy, x0, y0):
    pos_final_x = (25*vx)-10
    pos_final_y = (25*vy)-2
    return pos_final_x + x0, pos_final_y + y0

def posicion_de_la_recta_mas_cercana_al_enemigo(a, b, c, x0, y0):
    pos_x = (b * (b * x0 - a * y0) - a * c) / (a**2 + b**2)
    pos_y = (a * (-b * x0 + a * y0) - b * c) / (a**2 + b**2)
    return pos_x, pos_y

def calculate_reward(estado_actual, estado_anterior):
    global balas
    reward = -0.1  # Recompensa por defecto
    report = ""
    
    # 1. Penalizar si el jugador está quieto
    if estado_actual[28] == estado_anterior[28] and estado_actual[29] == estado_anterior[29]:
        report += "Jugador está quieto"
        reward -= 1
    
    # 2. Recompensa por entrar a una sala nueva (suma de vidas enemigas pasa de 0 a >0)
    if estado_anterior[31] == 0 and estado_actual[31] > 0:
        report += "Entró a una sala nueva. "
        reward += 100
    
    # 3. Recompensa por completar las 4 salas (todos los booleanos True)
    salas_completadas_actual = [estado_actual[33], estado_actual[34], estado_actual[35], estado_actual[36]]
    salas_completadas_anterior = [estado_anterior[33], estado_anterior[34], estado_anterior[35], estado_anterior[36]]
    if not all(salas_completadas_anterior) and all(salas_completadas_actual):
        report += "Todas las salas completadas. "
        reward += 20000
    
    # Preparar datos para cálculos posteriores
    player_x = estado_actual[28]
    player_y = estado_actual[29]
    
    # 4. Detectar enemigos vivos (posición != (0,0))
    enemigos = []
    for i in range(24, 28, 2):  # Índices 24-25 y 26-27
        if estado_actual[i] != 0 or estado_actual[i+1] != 0:
            enemigos.append((estado_actual[i], estado_actual[i+1]))
    
    # 5. Recompensa por disparar cerca de enemigo
    # Detectar lágrimas nuevas (transición de inactivas a activas)
    nuevas_lagrimas = []
    for i in range(0, 12, 4):  # Grupos de 4 elementos
        idx_ant = i
        idx_act = i
        
        # Verificar si la lágrima es nueva (inactiva antes, activa ahora)
        anterior_inactiva = all(v == 0 for v in estado_anterior[idx_ant:idx_ant+4])
        actual_activa = any(v != 0 for v in estado_actual[idx_act:idx_act+4])
        
        if anterior_inactiva and actual_activa:
            nuevas_lagrimas.append((
                estado_actual[idx_act],     # x
                estado_actual[idx_act+1],   # y
                estado_actual[idx_act+2],   # vx
                estado_actual[idx_act+3]    # vy
            ))
    
    # Verificar si hay enemigos cerca del jugador (<=10 unidades)
    enemigo_cerca = False
    for ex, ey in enemigos:
        distancia = math.sqrt((ex - player_x)**2 + (ey - player_y)**2)
        if distancia <= 60:
            enemigo_cerca = True
            break
    
    if nuevas_lagrimas and enemigo_cerca:
        report += "Disparó cerca de un enemigo. "
        reward += 2
    
    # 6. Recompensa por trayectoria de lágrima que intersecta enemigo
    for lagrima in nuevas_lagrimas:
        x0, y0, vx0, vy0 = lagrima
        for ex, ey in enemigos:
            dist_lagrima_enemigo_en_el_punto_mas_cercano_entre_los_dos, pos_x_en_la_recta_que_es_mas_cercana_al_enemigo, pos_y_en_la_recta_que_es_mas_cercana_al_enemigo = distancia_trayectoria(x0, y0, vx0, vy0, ex, ey)
            pos_x_predicha, pos_y_predicha = rango_predicho(vx0, vy0, x0, y0)
            dentro_del_rango_x = x0 <= pos_x_en_la_recta_que_es_mas_cercana_al_enemigo and pos_x_en_la_recta_que_es_mas_cercana_al_enemigo <= pos_x_predicha
            dentro_del_rango_y = y0 <= pos_y_en_la_recta_que_es_mas_cercana_al_enemigo and pos_y_en_la_recta_que_es_mas_cercana_al_enemigo <= pos_y_predicha
            dentro_del_rango = dentro_del_rango_x and dentro_del_rango_y
            chochan_hitbox = dist_lagrima_enemigo_en_el_punto_mas_cercano_entre_los_dos < 9

            if dentro_del_rango and chochan_hitbox:  # Hit detection
                report += f"Lágrima intersecta enemigo). "
                reward += 50
    
    # 7. Recompensa por matar enemigos (posición cambia de no-cero a (0,0))
    for i in range(24, 28, 2):
        idx_x = i
        idx_y = i+1
        # Verificar muerte (vivo antes, muerto ahora)
        if (estado_anterior[idx_x] != 0 or estado_anterior[idx_y] != 0) and \
           (estado_actual[idx_x] == 0 and estado_actual[idx_y] == 0):
            report += "Enemigo muerto"
            reward += 20
    
    # 8. Penalizar por balas que amenazan al jugador (XDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD)
    # Detectar balas nuevas
    bullet_index = 0

    print("balas antes de procesar:", balas)
    for i in range(12, 24, 4):  # Grupos de 4 elementos
        idx_ant = i
        idx_act = i
        print(f"Bullet index: {bullet_index}, bullet ID: {estado_anterior[37+bullet_index]}")
        
        # Verificar si la bala es nueva
        anterior_inactiva = all(v == 0 for v in estado_anterior[idx_ant:idx_ant+4])
        actual_activa = any(v != 0 for v in estado_actual[idx_act:idx_act+4])

        print(f"Anterior inactiva: {anterior_inactiva}, Actual activa: {actual_activa}")
        
        if anterior_inactiva and actual_activa:
            balas.append((
                estado_actual[idx_act],     # x
                estado_actual[idx_act+1],   # y
                estado_actual[idx_act+2],   # vx
                estado_actual[idx_act+3],    # vy
                estado_actual[37+bullet_index],  # indice de la bala
            ))

        if not anterior_inactiva and not actual_activa:
            # Si la bala ya no está activa, eliminarla de la lista
            print(f"Bala {bullet_index} ya no está activa, eliminando...")
            balas = [b for b in balas if b[4] != estado_anterior[37+bullet_index]]
        bullet_index += 1
    print("balas después de procesar:", balas)


    # Verificar si la trayectoria amenaza al jugador
    for bala in balas:
        x0, y0, vx0, vy0, _ = bala
        dist, _, _ = distancia_trayectoria(x0, y0, vx0, vy0, player_x, player_y)
        if dist < 9:  # Peligro de impacto
            report += "Bala amenaza al jugador. "
            reward -= 1
    
    # 9. Penalizar por pérdida de vida
    if estado_actual[30] < estado_anterior[30]:
        report += "Jugador perdió vida. "
        reward -= 20
    
    print(f"Report: {report.strip()}")
    print(f"Reward: {reward}")
    return reward

In [ ]:
prevState = []
currState = []

def parse_line(line: str, raw: bool = True):
    """
    Extrae los datos tras 'DATA:' y los organiza en un dict.
    """
    if 'DATA:' not in line:
        print("No se encontró 'DATA:' en la línea.")
        return {}
    # Separa el prefijo y la parte de datos

    frame_part = line.split('FRAME:')[1].split('DATA:')[0].strip()
    frame_number = int(frame_part)

    print(f"Frame: {frame_number}")

    _, raw = line.split('DATA:', 1)
    raw = raw.strip()
    parts = raw.split('|')

    if len(parts) != 40:
        raise ValueError(f"Esperaba 40 campos, encontré {len(parts)}")
    
    if raw:
        return [ # Convertir a float y int según corresponda
            float(parts[i]) if i < 30 else 
            int(float(parts[i])) if i in (30, 32) else 
            float(parts[i]) if i < 33 else 
            bool(int(float(parts[i]))) if i <= 36 else 
            int(float(parts[i]))
            for i in range(40)
        ]
    # Convierte los primeros 30 en float
    floats = list(map(float, parts[0:30]))
    # Campos 30,31,32 -> int
    player_life      = int(parts[30])
    enemies_life_sum = float(parts[31])
    room_id          = int(parts[32])
    # Campos 33–36 -> bool (0/1)
    rooms_completed = list(map(lambda x: bool(int(x)), parts[33:37]))
    
    # Organiza en estructuras más legibles
    tears  = [
        {'x': floats[i],   'y': floats[i+1],
         'vx': floats[i+2],'vy': floats[i+3]}
        for i in (0, 4, 8)
    ]
    bullets = [
        {'x': floats[i],   'y': floats[i+1],
         'vx': floats[i+2],'vy': floats[i+3]}
        for i in (12, 16, 20)
    ]
    enemies = [
        {'x': floats[24], 'y': floats[25]},
        {'x': floats[26], 'y': floats[27]},
    ]
    player = {'x': floats[28], 'y': floats[29], 'life': player_life}
    
    return {
        'tears': tears,
        'bullets': bullets,
        'enemies': enemies,
        'player': player,
        'enemies_life_sum': enemies_life_sum,
        'room_id': room_id,
        'rooms_completed': {
            'room_1': rooms_completed[0],
            'room_2': rooms_completed[1],
            'room_3': rooms_completed[2],
            'room_4': rooms_completed[3],
        }
    }

def follow(file_path: str, delay: float = 1/60):
    """
    Abre el archivo y espera líneas nuevas. Cada vez que llega una,
    la parsea y la imprime (o puedes procesarla como prefieras).
    """
    global prevState, currState
    with open(file_path, 'r', encoding='utf-8') as f:
        # Ir al final para no procesar histórico
        f.seek(0, 2)
        while True:
            line = f.readline()
            if not line:
                time.sleep(delay)
                continue
            try:
                data = parse_line(line)
                if data:
                    if prevState == [] and currState == []:
                        # Si es la primera línea, simplemente guarda el estado
                        prevState = data
                    elif prevState != [] and currState == []:
                        # Si es la segunda línea, guarda el estado actual
                        currState = data
                    else:
                        prevState = currState
                        currState = data
                        calculate_reward(currState, prevState)

            except ValueError as e:
                # En caso de línea corrupta o formato inesperado
                print(f"Error parseando línea: {e}")

if __name__ == '__main__':
    # Cambia la ruta por la de tu archivo .txt

    try:
        follow(r'C:\Users\erneset\Documents\My Games\Binding of Isaac Repentance+\log.txt')


    except KeyboardInterrupt:
            print("👋 ¡Hasta luego!")



Frame: 264255
Frame: 264256
Frame: 264257
balas antes de procesar: []
Bullet index: 0, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 1, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 2, bullet ID: 0
Anterior inactiva: True, Actual activa: False
balas después de procesar: []
Report: 
Reward: -0.1
Frame: 264258
balas antes de procesar: []
Bullet index: 0, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 1, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 2, bullet ID: 0
Anterior inactiva: True, Actual activa: False
balas después de procesar: []
Report: 
Reward: -0.1
Frame: 264259
balas antes de procesar: []
Bullet index: 0, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 1, bullet ID: 0
Anterior inactiva: True, Actual activa: False
Bullet index: 2, bullet ID: 0
Anterior inactiva: True, Actual activa: False
balas después de procesar: []
Report: 
Reward: -0.1
Frame: